# Import Library

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
from utils.preprocessing import cleaningText
from utils.preprocessing import casefoldingText
from utils.preprocessing import stemmingText
from utils.preprocessing import filteringText
from sentence_transformers import SentenceTransformer, util
from sqlalchemy import create_engine

import pandas as pd

# Load model
model = SentenceTransformer('distiluse-base-multilingual-cased')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\ASUS\Desktop\arkanesia-dataset\arkanesia_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [13]:
# Koneksi ke MySQL
engine = create_engine(os.getenv("DATABASE_URL"))

# Query SQL
query = "SELECT * FROM tour"

# Baca data ke DataFrame
wisata_df = pd.read_sql(query, engine)

wisata_df.head()

,id,name,date,location,image,description,link,prices,district,province,latitude,longitude,createdAt,updatedAt
0,1,Pantai Kuta,1936-01-01 00:00:00,"Kuta, Badung",https://labirutour.com/wp-content/uploads/2020...,Pantai Kuta adalah salah satu destinasi wisata...,https://www.google.com/url?sa=i&url=https%3A%2...,0.0,Kuta,Bali,-8.7177,115.1682,2025-04-14 11:12:31.346,2025-04-14 11:12:31.346
1,2,Pura Ulun Danu Beratan,1633-01-01 00:00:00,"Bedugul, Tabanan",https://awsimages.detik.net.id/community/media...,Pura Ulun Danu Beratan adalah pura air yang te...,https://www.google.com/url?sa=i&url=https%3A%2...,50000.0,Baturiti,Bali,-8.2750,115.1675,2025-04-14 11:12:31.355,2025-04-14 11:12:31.355
2,3,Tegallalang Rice Terrace,1800-01-01 00:00:00,"Tegallalang, Gianyar",https://media.cntraveler.com/photos/5e35bdf00e...,Tegallalang Rice Terrace terkenal dengan peman...,https://www.google.com/url?sa=i&url=https%3A%2...,25000.0,Tegallalang,Bali,-8.4366,115.2790,2025-04-14 11:12:31.360,2025-04-14 11:12:31.360
3,4,Gunung Batur,1804-01-01 00:00:00,"Kintamani, Bangli",https://upload.wikimedia.org/wikipedia/commons...,Gunung Batur adalah gunung berapi aktif yang p...,https://www.google.com/url?sa=i&url=https%3A%2...,30000.0,Kintamani,Bali,-8.2390,115.3750,2025-04-14 11:12:31.366,2025-04-14 11:12:31.366
4,5,Pantai Pandawa,2011-01-01 00:00:00,"Kutuh, Badung",https://cozzy.id/uploads/0000/630/2024/09/04/c...,Pantai Pandawa dikenal dengan pasir putihnya y...,https://www.google.com/url?sa=i&url=https%3A%2...,15000.0,Kutuh,Bali,-8.8486,115.1889,2025-04-14 11:12:31.371,2025-04-14 11:12:31.371


# Explore Data

In [4]:
wisata_df.duplicated().sum()

np.int64(0)

In [5]:
wisata_df.isnull().sum()

id             0
name           0
date           0
location       0
image          0
description    0
link           1
prices         0
district       0
province       0
latitude       0
longitude      0
createdAt      0
updatedAt      0
dtype: int64

In [6]:
wisata_df.describe()

,id,prices,latitude,longitude,createdAt,updatedAt
count,100.000000,100.000000,100.000000,100.000000,100,100
mean,50.500000,25160.000000,-3.726152,112.452892,2025-04-14 11:12:32.564790016,2025-04-14 11:12:32.564790016
min,1.000000,0.000000,-10.879400,95.317100,2025-04-14 11:12:31.346000,2025-04-14 11:12:31.346000
25%,25.750000,5000.000000,-7.712775,105.583175,2025-04-14 11:12:31.760499968,2025-04-14 11:12:31.760499968
50%,50.500000,15000.000000,-5.465600,110.390700,2025-04-14 11:12:32.408500224,2025-04-14 11:12:32.408500224
75%,75.250000,26250.000000,0.649775,118.747425,2025-04-14 11:12:33.154749952,2025-04-14 11:12:33.154749952
max,100.000000,150000.000000,5.840100,140.705600,2025-04-14 11:12:34.325000,2025-04-14 11:12:34.325000
std,29.011492,32759.830046,4.413499,10.479746,NaN,NaN


In [7]:
wisata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           100 non-null    int64         
 1   name         100 non-null    object        
 2   date         100 non-null    object        
 3   location     100 non-null    object        
 4   image        100 non-null    object        
 5   description  100 non-null    object        
 6   link         99 non-null     object        
 7   prices       100 non-null    float64       
 8   district     100 non-null    object        
 9   province     100 non-null    object        
 10  latitude     100 non-null    float64       
 11  longitude    100 non-null    float64       
 12  createdAt    100 non-null    datetime64[ns]
 13  updatedAt    100 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(8)
memory usage: 11.1+ KB


# Preprocessing Data

## Cleaning Data

In [15]:
wisata_cleaned_df = pd.DataFrame(wisata_df)
wisata_cleaned_df['name'] = wisata_cleaned_df['name'].apply(cleaningText)
wisata_cleaned_df['location'] = wisata_cleaned_df['location'].apply(cleaningText)
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(cleaningText)
wisata_cleaned_df['district'] = wisata_cleaned_df['district'].apply(cleaningText)
wisata_cleaned_df['province'] = wisata_cleaned_df['province'].apply(cleaningText)

## Case Folding Data

In [16]:
wisata_cleaned_df['name'] = wisata_cleaned_df['name'].apply(casefoldingText)
wisata_cleaned_df['location'] = wisata_cleaned_df['location'].apply(casefoldingText)
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(casefoldingText)
wisata_cleaned_df['district'] = wisata_cleaned_df['district'].apply(casefoldingText)
wisata_cleaned_df['province'] = wisata_cleaned_df['province'].apply(casefoldingText)
wisata_cleaned_df.head()

,id,name,date,location,image,description,link,prices,district,province,latitude,longitude,createdAt,updatedAt
0,1,pantai kuta,1936-01-01 00:00:00,kuta badung,https://labirutour.com/wp-content/uploads/2020...,pantai kuta adalah salah satu destinasi wisata...,https://www.google.com/url?sa=i&url=https%3A%2...,0.0,kuta,bali,-8.7177,115.1682,2025-04-14 11:12:31.346,2025-04-14 11:12:31.346
1,2,pura ulun danu beratan,1633-01-01 00:00:00,bedugul tabanan,https://awsimages.detik.net.id/community/media...,pura ulun danu beratan adalah pura air yang te...,https://www.google.com/url?sa=i&url=https%3A%2...,50000.0,baturiti,bali,-8.2750,115.1675,2025-04-14 11:12:31.355,2025-04-14 11:12:31.355
2,3,tegallalang rice terrace,1800-01-01 00:00:00,tegallalang gianyar,https://media.cntraveler.com/photos/5e35bdf00e...,tegallalang rice terrace terkenal dengan peman...,https://www.google.com/url?sa=i&url=https%3A%2...,25000.0,tegallalang,bali,-8.4366,115.2790,2025-04-14 11:12:31.360,2025-04-14 11:12:31.360
3,4,gunung batur,1804-01-01 00:00:00,kintamani bangli,https://upload.wikimedia.org/wikipedia/commons...,gunung batur adalah gunung berapi aktif yang p...,https://www.google.com/url?sa=i&url=https%3A%2...,30000.0,kintamani,bali,-8.2390,115.3750,2025-04-14 11:12:31.366,2025-04-14 11:12:31.366
4,5,pantai pandawa,2011-01-01 00:00:00,kutuh badung,https://cozzy.id/uploads/0000/630/2024/09/04/c...,pantai pandawa dikenal dengan pasir putihnya y...,https://www.google.com/url?sa=i&url=https%3A%2...,15000.0,kutuh,bali,-8.8486,115.1889,2025-04-14 11:12:31.371,2025-04-14 11:12:31.371


### Stemming Text

In [17]:
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(stemmingText)
wisata_cleaned_df.head()

,id,name,date,location,image,description,link,prices,district,province,latitude,longitude,createdAt,updatedAt
0,1,pantai kuta,1936-01-01 00:00:00,kuta badung,https://labirutour.com/wp-content/uploads/2020...,pantai kuta adalah salah satu destinasi wisata...,https://www.google.com/url?sa=i&url=https%3A%2...,0.0,kuta,bali,-8.7177,115.1682,2025-04-14 11:12:31.346,2025-04-14 11:12:31.346
1,2,pura ulun danu beratan,1633-01-01 00:00:00,bedugul tabanan,https://awsimages.detik.net.id/community/media...,pura ulun danu rat adalah pura air yang letak ...,https://www.google.com/url?sa=i&url=https%3A%2...,50000.0,baturiti,bali,-8.2750,115.1675,2025-04-14 11:12:31.355,2025-04-14 11:12:31.355
2,3,tegallalang rice terrace,1800-01-01 00:00:00,tegallalang gianyar,https://media.cntraveler.com/photos/5e35bdf00e...,tegallalang rice terrace kenal dengan pandang ...,https://www.google.com/url?sa=i&url=https%3A%2...,25000.0,tegallalang,bali,-8.4366,115.2790,2025-04-14 11:12:31.360,2025-04-14 11:12:31.360
3,4,gunung batur,1804-01-01 00:00:00,kintamani bangli,https://upload.wikimedia.org/wikipedia/commons...,gunung batur adalah gunung rap aktif yang popu...,https://www.google.com/url?sa=i&url=https%3A%2...,30000.0,kintamani,bali,-8.2390,115.3750,2025-04-14 11:12:31.366,2025-04-14 11:12:31.366
4,5,pantai pandawa,2011-01-01 00:00:00,kutuh badung,https://cozzy.id/uploads/0000/630/2024/09/04/c...,pantai pandawa kenal dengan pasir putih yang b...,https://www.google.com/url?sa=i&url=https%3A%2...,15000.0,kutuh,bali,-8.8486,115.1889,2025-04-14 11:12:31.371,2025-04-14 11:12:31.371


### Filtering text

In [15]:
# wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(filteringText)
# wisata_cleaned_df.head()

## Buat Embeddings

In [18]:
wisata_cleaned_df['combined'] = (
    wisata_cleaned_df['name'] + ' ' +
    wisata_cleaned_df['district'] + ' ' +
    wisata_cleaned_df['province'] + ' ' +
    wisata_cleaned_df['location'] + ' ' +
    wisata_cleaned_df['description']
)

In [19]:
# Buat embeddings
embeddings = model.encode(wisata_cleaned_df['combined'].tolist(), convert_to_tensor=True)

In [28]:
def rekomendasi_wisata(pertanyaan_user, top_k=3):
    original_question = pertanyaan_user.lower()

    all_provinces = wisata_cleaned_df['province'].str.lower().unique()

    lokasi_filter = None
    for prov in all_provinces:
        if prov in original_question:
            lokasi_filter = prov
            break

    if lokasi_filter:
        df_filtered = wisata_cleaned_df[wisata_cleaned_df['province'].str.lower().str.contains(lokasi_filter)]
        if df_filtered.empty:
            print(f"Tidak ada data wisata di lokasi '{lokasi_filter.title()}', menampilkan hasil umum...\n")
            df_filtered = wisata_cleaned_df
    else:
        df_filtered = wisata_cleaned_df

    pertanyaan_user = cleaningText(pertanyaan_user)
    pertanyaan_user = casefoldingText(pertanyaan_user)
    pertanyaan_user = stemmingText(pertanyaan_user)

    query_embedding = model.encode(pertanyaan_user, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]

    top_results = cos_scores.topk(k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        idx = idx.item()
        row = df_filtered.iloc[idx]
        print(f"{row['name']} ({row['province']}) - Skor: {score:.4f}")
        print(f"Deskripsi: {row['description']}")
        print("-" * 50)


In [32]:
rekomendasi_wisata("pantai jawa")

pantai parangtritis (yogyakarta) - Skor: 0.3542
Deskripsi: pantai parangtritis adalah pantai yang kenal dengan ombak besar dan legenda nyi roro kidul tawar pandang matahari benam yang pukau
--------------------------------------------------
pantai mutun (lampung) - Skor: 0.3194
Deskripsi: wisata pantai populer di lampung letak di camat padang cermin kabupaten sawar pantai ini kenal dengan pasir putih yang lembut dan air laut yang jernih cocok untuk renang main air atau sekadar santa di tepi pantai keliling oleh bukitbukit hijau pantai mutun tawar pandang alam yang tenang dan udara segar yang jauh dari ramai kota lokasi yang hanya jarak sekitar km dari bandar lampung buat mudah jangkau baik dengan kendara pribadi maupun transportasi umum selain indah alam pantai mutun juga lengkap dengan bagai fasilitas wisata seperti gazebo warung makan seafood dan area main anak unjung dapat nikmat hidang laut segar sambil nikmat pandang teluk lampung salah satu daya tarik utama pantai ini adalah pula